# Event Planning Chatbot using Deep Learning 


### Importing the necessary module and package

In [77]:
import numpy as np
import json
import pickle
import random
import nltk
import numpy
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.layers import Dense, Dropout

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding, Bidirectional
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

### Import the dataset 

In [78]:
data_file=open('dataset1.json').read()
data=json.loads(data_file)

### Preprocessing the dataset or tokenize the dataset

In [79]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

words = [] # words are used in the dataset
classes = [] # add tag only in it 
documents = [] # Make list of words or patterns with the tag (['Hello'], 'greeting'), (['Hey'], 'greeting')

# Extract words and classes from the dataset
for intent in data['intents']:
    for pattern in intent['patterns']:
        words.extend(nltk.word_tokenize(pattern))
        documents.append((nltk.word_tokenize(pattern), intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize words and remove ignore characters
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ['?', '!', ',', "'s"]] #remove the special character and es
words = sorted(list(set(words)))    # remove the duplicate
classes = sorted(list(set(classes)))  #remove the duplicate tag



### Prepare the training dataset from words and documents

In [80]:
# Prepare training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    #print(pattern_words)
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    #bag is the input array of 0,1. 0 reps 


### Split the dataset

In [81]:
random.shuffle(training)


# Split features and labels
X_train = np.array([bag for bag, _ in training])
y_train = np.array([output_row for _, output_row in training])

print("Training data shape:", X_train.shape)
print("Label data shape:", y_train.shape)


Training data shape: (95, 128)
Label data shape: (95, 29)


## Create a Model using LSTM

In [82]:

"""
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))
# accuracy = 100%
"""
model = Sequential()
model.add(Embedding(input_dim=len(X_train[0]), output_dim=128))
model.add(LSTM(110, return_sequences = True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(110))
model.add(Dense(208, activation = 'relu'))
model.add(Dense(len(y_train[0]), activation='softmax'))
# accuracy = 98%


### Compile , Train and Evaluate the model 

In [70]:
# Compile model
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


# Train model
weights = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=10, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_train, y_train)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

print("Training completed!")

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 131ms/step - accuracy: 0.0250 - loss: 3.3735
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.0140 - loss: 3.3672
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.0374 - loss: 3.3638
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.0488 - loss: 3.3589
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.0276 - loss: 3.3431
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.0281 - loss: 3.3438
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.0452 - loss: 3.3680
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.0515 - loss: 3.3426
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.0527 - loss: 3.3504
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.0741 - loss: 3.3392
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.0737 - loss: 3.3447
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1

## Chat with the event planning chatbot (BAKAS)

##### use quit to exit the chat

In [13]:
while True:
    input_text = input("You: ")
    if input_text.lower() == 'quit':
        break
    else:
        input_words = nltk.word_tokenize(input_text)
        input_bow = [0] * len(words)
        for w in input_words:
            if w.lower() in words:
                input_bow[words.index(w.lower())] = 1

        input_bow = np.array(input_bow).reshape(1, -1)
        predicted_probabilities = model.predict(input_bow)
        predicted_class_index = np.argmax(predicted_probabilities)
        
        
        def get_response(predicted_tag):
            for intent in data['intents']:
                if intent['tag'] == predicted_tag:
                    return random.choice(intent['responses'])
            return "Sorry, I don't understand that."
            
        predicted_tag = classes[predicted_class_index] # Replace with the predicted tag from your model
        response = get_response(predicted_tag)   
        print("Bot:",response)

You:  hall


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: Glad I could make you laugh !


You:  quit
